In [31]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
from obspy.core.event import Catalog
from obspy.core.event import Event
from obspy.core.event import Origin
from obspy.core.event import Magnitude

# Read and create dictionary

In [2]:
directory = os.fsencode('flegrei_2014_2023_GOSSIP')
print('reading and saving:')
cat_all={}
for file in os.listdir(directory):
    filename = os.fsdecode(file)

    if filename.endswith(".csv"): 
        print(filename)

        with open('flegrei_2014_2023_GOSSIP/'+filename, newline='') as f:
            first_line=True
            reader = csv.reader(f, delimiter=',')
            cat={}
            for row in reader:
                if first_line==True:
                    first_line=False
                    for key in row:
                        cat.setdefault(key,[])
                        cat_all.setdefault(key,[])
                else:
                        for n,key in enumerate(cat):
                             cat[key].append(row[n])
                             cat_all[key].append(row[n])

        del cat['Area']
        del cat['Type']
        del cat_all['Area']
        del cat_all['Type']           
        #save
        np.save('flegrei_2014_2023_GOSSIP/'+ filename[:-4] +'.npy', cat)
    else:
        continue
#save
np.save('flegrei_2014_2023_GOSSIP/events_all.npy', cat_all)
print('events_all')

reading and saving:
events_2017.csv
events_2016.csv
events_2014.csv
events_2015.csv
events_2018.csv
events_2019.csv
events_2022.csv
events_2023.csv
events_2021.csv
events_2020.csv
events_all


## load all dictionaries

In [3]:
'''
print('loading dictionaries:')

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
    if filename.endswith(".npy"): 
        print(filename)

        var_name='cat' + filename[6:-4]
        cat=np.load('flegrei_2014_2023_GOSSIP/'+ filename , allow_pickle='TRUE').item()
        exec(f"{var_name} = {cat}")
        
    else:
        continue
'''

loading dictionaries:
events_2014.npy
events_2015.npy
events_2017.npy
events_2016.npy
events_2018.npy
events_2019.npy
events_2021.npy
events_2020.npy
events_2022.npy
events_2023.npy
events_all.npy


## load big dictionary (catalogue)

In [4]:
cat_all=np.load('flegrei_2014_2023_GOSSIP/events_all.npy' , allow_pickle='TRUE').item()

# CODE

In [4]:
print(type(cat_all))
print(cat_all.keys())
print(len(cat_all['Longitude']))

<class 'dict'>
dict_keys(['#EventID', 'Time', 'Latitude', 'Longitude', 'Depth/km', 'MagType', 'Magnitude', 'MagErr', 'Level'])
15178


In [7]:
#not working for strings
'''
plt.figure()
x=cat_all['Longitude'][0:50]
y=cat_all['Latitude'][0:50]
plt.plot(x,y,'.b')
'''

"\nplt.figure()\nx=cat_all['Longitude'][0:50]\ny=cat_all['Latitude'][0:50]\nplt.plot(x,y,'.b')\n"

In [8]:
print(cat_all.keys())
print(cat_all['#EventID'][1],',',cat_all['Time'][1],',',cat_all['Latitude'][1],',',cat_all['Longitude'][1],',',cat_all['Depth/km'][1],',',cat_all['MagType'][1],',',cat_all['Magnitude'][1],',',cat_all['MagErr'][1],',',cat_all['Level'][1])

dict_keys(['#EventID', 'Time', 'Latitude', 'Longitude', 'Depth/km', 'MagType', 'Magnitude', 'MagErr', 'Level'])
8499 , 2017-01-16 04:18:16.43 ,  ,  ,  , D , -1.1 , 0.3 , Bollettino


## create two catalogues: 1) only magnitudes ; 2) magnitudes and location

# !!!not working!!!

In [ ]:
cat_mag=dict.fromkeys(cat_all.keys(),[])
cat_loc=dict.fromkeys(cat_all.keys(),[])

for n,ev_id in enumerate(cat_all['#EventID']):
    if ( cat_all['Latitude'][n]=='' and cat_all['MagType'][n]=='' ):
        continue
    elif ( cat_all['Latitude'][n]=='' and cat_all['MagType'][n]!='' ):
        for key in cat_mag:
            cat_mag[key][n].append(cat_all[key][n])
    elif ( cat_all['Latitude'][n]!='' and cat_all['MagType'][n]!='' ):
        for key in cat_loc:
            cat_loc[key][n].append(cat_all[key][n])
    else:
        print('event not included in catalogue')

# create catalogue obspy

In [25]:
cat_obs=Catalog()
for ind,id_number in enumerate(cat_all['#EventID']):
    origin=Origin()
    magnitude=Magnitude()
    
    origin.resource_id=id_number
    
    origin.time=cat_all['Time'][ind]
    
    try:
        origin.latitude=float(cat_all['Latitude'][ind])
    except:
        None
    
    try:
        origin.longitude=float(cat_all['Longitude'][ind])
    except:
        None
    
    try:
        origin.depth=float(cat_all['Depth/km'][ind])
    except:
        None
        
    event=Event(resource_id=id_number)
    event.origins.append(origin)
    event.magnitudes.append(magnitude)
    cat_obs.events.append(event)


In [30]:
#cat_obs.events[0].origins[0].__dict__
cat_obs.events[0].magnitudes


[]